In [1]:
%load_ext autoreload
%matplotlib nbagg
%autoreload 2

In [2]:
# To use plotly inside jupyter
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [21]:
import os.path as osp
from geo_tool import Mesh
from scipy.io.matlab import savemat
import numpy as np
from geo_tool.solids.plotting import plot_mesh_via_plotly
from general_tools.in_out.basics import create_dir

In [4]:
import helper
from tf_lab.diff_maps.in_out import raw_data, produce_net_data,\
prep_splits_labels_for_task, produce_diff_maps, classes_of_tasks

In [28]:
top_part_dir = '/orions4-zfs/projects/optas/DATA/OUT/latent_diff_maps/experiments/SCAPE_8_poses/body_parts'
in_f = osp.join(top_part_dir, 'body_parts.npz')
parts = np.load(in_f)
left_arm = parts['left_arm']
split_vertical = parts['split_vertical']
left_leg = parts['left_leg']
right_leg = parts['right_leg']
upper_torso = parts['upper_torso']

out_f = osp.join(top_part_dir, 'body_parts')
savemat(out_f, parts)

In [27]:
mesh_file = '/orions4-zfs/projects/optas/DATA/Meshes/SCAPE_8_poses/Shape1024.off'
mesh = Mesh(file_name = mesh_file)
plot_mesh_via_plotly(mesh, vertex_color=left_leg)

In [6]:
top_mesh_dir = '/orions4-zfs/projects/optas/DATA/Meshes/SCAPE_8_poses/'
gt_param_f = osp.join(top_mesh_dir, 'gt_shape_params.mat')
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/latent_diff_maps/experiments/SCAPE_8_poses/'

In [20]:
total_shapes = helper.total_shapes
n_pose_classes = helper.n_pose_classes

knn = 0
n_cons = 40
n_pc_points = 2048
sub_member_per_class = 50
n_shapes = sub_member_per_class * n_pose_classes

top_experiment_dir = osp.join(top_data_dir, '50_extract_%d_knn_50_fmapd' % (knn,) )

In [24]:
synced_bases_file = osp.join(top_experiment_dir, 'synconized_basis.mat')
shapes_with_eigen = osp.join(top_experiment_dir, 'shapes_with_eigen.mat')

gt_latent_params_2, in_pcs_2, pose_labels_2 = \
raw_data(top_mesh_dir, gt_param_f, sub_member_per_class, n_pc_points)

diff_maps = produce_diff_maps(synced_bases_file, n_cons, n_shapes)

In [26]:
from scipy.io import loadmat
import hdf5storage

In [27]:
shapes_with_eigen = hdf5storage.loadmat(shapes_with_eigen)

In [46]:
a = shapes_with_eigen['shapes']

In [86]:
a[0][0][0]['evecs'].shape

(6449, 50)

In [88]:
a[0][0][0]['A'].shape

(1,)

In [45]:
# shapes_with_eigen['shapes'][0][0][0][0]
# print shapes_with_eigen.keys()

# loadmat(shapes_with_eigen)

TypeError: coercing to Unicode: need string or buffer, dict found

In [43]:
import scipy.io as spio


In [44]:
def print_mat_nested(d, indent=0, nkeys=0):
    """Pretty print nested structures from .mat files   
    Inspired by: `StackOverflow <http://stackoverflow.com/questions/3229419/pretty-printing-nested-dictionaries-in-python>`_
    """
    
    # Subset dictionary to limit keys to print.  Only works on first level
    if nkeys>0:
        d = {k: d[k] for k in d.keys()[:nkeys]}  # Dictionary comprehension: limit to first nkeys keys.

    if isinstance(d, dict):
        for key, value in d.iteritems():         # iteritems loops through key, value pairs
          print '\t' * indent + 'Key: ' + str(key)
          print_mat_nested(value, indent+1)

    if isinstance(d,np.ndarray) and d.dtype.names is not None:  # Note: and short-circuits by default
        for n in d.dtype.names:    # This means it's a struct, it's bit of a kludge test.
            print '\t' * indent + 'Field: ' + str(n)
            print_mat_nested(d[n], indent+1)
            
            

In [41]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    
    from: `StackOverflow <http://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries>`_
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

In [ ]:
# n_shapes = len(in_diffs['ucb'])
# diff_dims = in_diffs['ucb'][1][0].shape
# temp = np.zeros(shape=(n_shapes, ) + diff_dims )
# for i in xrange(n_shapes):
#     temp[i] = in_diffs['ucb'][i][0]
# in_diffs = temp    
# return in_diffs


# a = loadmat(shapes_with_eigen)
# synced_bases = synced_bases['sync_bases']
# synced_bases = synced_bases[:, :n_cons]